In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
import numpy as np
import glob
import os
from IPython.display import Image,display

In [ ]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')

In [ ]:
bbox.shape[0]

In [ ]:
import matplotlib.patches as patches
from PIL import Image
import numpy as np

#i = np.random.randint(bbox.shape[0])

i=8873
im = np.array(Image.open('Data/DeepFashion/'+bbox['image_name'][i]), dtype=np.uint8)

# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(im)

lx = bbox['x_1'][i]
ly = bbox['y_1'][i]
w = bbox['x_2'][i] - lx
h = bbox['y_2'][i] - ly

# Create a Rectangle patch
rect = patches.Rectangle((lx,ly),w,h,linewidth=1,edgecolor='r',facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

In [ ]:
def cropDF(bbox):
    im = np.array(PIL.Image.open('Data/DeepFashion/'+bbox['image_name']), dtype=np.uint8)
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    return im[ly:uy,lx:ux]

In [ ]:
fig,ax = plt.subplots(1)

ax.imshow(cropDF(bbox.iloc[56]))
plt.show()

In [ ]:
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
import numpy as np
import glob
import os

In [ ]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input
model = VGG16(weights='imagenet')

In [ ]:
def crop_resize_DF(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

In [ ]:
fig,ax = plt.subplots(1)

ax.imshow(crop_resize_DF(bbox.iloc[56]))
plt.show()

In [ ]:
def predictLabel(bbox):
    image = crop_resize_DF(bbox)
    file = 'Data/DeepFashion/'+bbox['image_name']
    image = np.expand_dims(image,axis=0)

    image = preprocess(image)
    preds = model.predict(image)
    P = imagenet_utils.decode_predictions(preds)
    display(Image(filename=file,width=inputShape[0],height=inputShape[1]))
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        # other way
        print(f'{i+1}. {label}: {prob*100:.2f}%')

In [ ]:
i = np.random.randint(bbox.shape[0])
predictLabel(bbox.iloc[i])

In [ ]:
def get_features(model,bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    image = preprocess(image)
    feature = model.predict(image)
    return np.squeeze(feature)

In [ ]:
def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

In [ ]:
fc6_model = Model(inputs=model.input,outputs=model.get_layer('fc1').output)

In [ ]:
N = 1000
subset = np.random.randint(bbox.shape[0],size=N)

In [ ]:
import time

start = time.time()
proc_imgs = []
for i in subset:
    proc_imgs.append(np.squeeze(preprocess_DF(bbox.iloc[i])))
proc_imgs = np.array(proc_imgs)
print(f'{time.time()-start:.2f} s')

In [ ]:
start = time.time()
feature_matrix = fc6_model.predict(proc_imgs)
print(f'{time.time()-start:.2f} s')

In [ ]:
from PIL import ImageDraw
def draw_bbox(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name']).convert('RGBA')
    
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    rect = PIL.Image.new('RGBA', img.size, (255,255,255,0))
    d = ImageDraw.Draw(img)
    d.rectangle(((lx,ly), (ux,uy)), outline="red")
    return PIL.Image.alpha_composite(img,rect)

In [ ]:
def find_match(i,image_df,model,feature_matrix):
    img_feature = model.predict(preprocess_DF(image_df.iloc[i]))
    L2 = np.sum((feature_matrix-img_feature)**2,axis=1)
    match = np.argmin(L2)

    print(f'distance: {np.sqrt(np.min(L2))}')
    display(Image(filename='Data/DeepFashion/'+image_df.iloc[i]['image_name'],
                  width=inputShape[0],height=inputShape[1]))
    display(Image(filename='Data/DeepFashion/'+image_df.iloc[match]['image_name'],
                  width=inputShape[0],height=inputShape[1]))
    
def gen_pairs(N,image_df,feature_matrix):
    img_index = np.random.randint(image_df.shape[0],size=N)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        L2 = np.sum((feature_matrix-img_feature)**2,axis=1)
        match = np.argsort(L2)[1]

        print(f'distance: {np.sqrt(L2[match])}')
        display(draw_bbox(image_df.iloc[i]))
        display(draw_bbox(image_df.iloc[match]))

In [ ]:
gen_pairs(3,bbox.iloc[subset],feature_matrix)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
fc6_embed = tsne.fit_transform(feature_matrix)

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# Plots an image at each x and y location. 
def plotImage(x, y, image, ax):
    im = OffsetImage(load_img(image,target_size=inputShape), zoom=0.15)
    ab = AnnotationBbox(im, (x, y), xycoords='data', frameon=False)
    ax.add_artist(ab)

# Create figure
def plot_tsne(N,embed,image_df,file):
    fig = plt.figure(figsize=(20,20),dpi=300)
    ax = fig.add_subplot(111)

    for i in range(N):
        plotImage(embed[i,0],embed[i,1],
                  'Data/DeepFashion/'+image_df.iloc[i]['image_name'],ax)

    ax.set_ylim(np.min(embed[:N,1]),np.max(embed[:N,1]))
    ax.set_xlim(np.min(embed[:N,0]),np.max(embed[:N,0]))

    plt.savefig(file)

In [ ]:
plot_tsne(fc6_embed,bbox.iloc[subset],'DF_1000_tsne.png')

In [ ]:
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')

In [ ]:
tops = bbox.iloc[cat[(cat.category_label>=1) & (cat.category_label<=20)].index]

In [ ]:
N = 2000
subset = np.random.randint(tops.shape[0],size=N)
proc_imgs = []
for i in subset:
    proc_imgs.append(np.squeeze(preprocess_DF(tops.iloc[i])))
proc_imgs = np.array(proc_imgs)

In [ ]:
start = time.time()
feature_matrix_tops = fc6_model.predict(proc_imgs)
print(f'{time.time()-start:.2f} s')

In [ ]:
gen_pairs(3,tops.iloc[subset],feature_matrix_tops)

In [ ]:
fc6_tops_embed = tsne.fit_transform(feature_matrix_tops)

In [ ]:
plot_tsne(500,fc6_tops_embed,tops.iloc[subset],'output/DF_tops_500_tsne.png')

In [ ]:
tops = bbox.iloc[cat[(cat.category_label>=1) & (cat.category_label<=20)].index]
bottoms = bbox.iloc[cat[(cat.category_label>=21) & (cat.category_label<=36)].index]
dresses = bbox.iloc[cat[(cat.category_label>36)].index]

In [ ]:
N = 10000
subset_tops = np.random.randint(tops.shape[0],size=N)
proc_tops = []
for i in subset_tops:
    proc_tops.append(np.squeeze(preprocess_DF(tops.iloc[i])))
proc_tops = np.array(proc_tops)

In [ ]:
N = 10000
subset_bottoms = np.random.randint(bottoms.shape[0],size=N)
proc_bottoms = []
for i in subset_bottoms:
    proc_bottoms.append(np.squeeze(preprocess_DF(bottoms.iloc[i])))
proc_bottoms = np.array(proc_bottoms)

In [ ]:
N = 10000
subset_dresses = np.random.randint(dresses.shape[0],size=N)
proc_dresses = []
for i in subset_dresses:
    proc_dresses.append(np.squeeze(preprocess_DF(dresses.iloc[i])))
proc_dresses = np.array(proc_dresses)

In [ ]:
import pickle
pickle.dump(tops.iloc[subset_tops] ,open('Data/feature_matrix/tops_10000_df.p','wb'))
pickle.dump(bottoms.iloc[subset_bottoms] ,open('Data/feature_matrix/bottoms_10000_df.p','wb'))
pickle.dump(dresses.iloc[subset_dresses] ,open('Data/feature_matrix/dresses_10000_df.p','wb'))

In [ ]:
from keras.layers import Dense

model = VGG16(weights='imagenet')
FC6_relu = Model(inputs=model.input,outputs=model.get_layer('fc1').output)
FC8_linear = Model(inputs = model.input,
                   output = Dense(1000, activation=None, name='FC8')(model.get_layer('predictions').input))

In [ ]:
start = time.time()
fc6_tops = FC6_relu.predict(proc_tops)
pickle.dump(FC6_relu_tops,open('Data/feature_matrix/FC6_relu_tops.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
pickle.dump(fc6_tops,open('Data/feature_matrix/FC6_relu_tops.p','wb'))

In [ ]:
start = time.time()
fc8_tops = FC8_linear.predict(proc_tops)
pickle.dump(fc8_tops,open('Data/feature_matrix/FC8_linear_tops.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
start = time.time()
fc6_bottoms = FC6_relu.predict(proc_bottoms)
pickle.dump(fc6_bottoms,open('Data/feature_matrix/FC6_relu_bottoms.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
start = time.time()
fc8_bottoms = FC8_linear.predict(proc_bottoms)
pickle.dump(fc8_bottoms,open('Data/feature_matrix/FC8_linear_bottoms.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
start = time.time()
fc6_dresses = FC6_relu.predict(proc_dresses)
pickle.dump(FC6_relu_dresses,open('Data/feature_matrix/FC6_relu_dresses.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
start = time.time()
fc8_dresses = FC8_linear.predict(proc_dresses)
pickle.dump(FC8_linear_dresses,open('Data/feature_matrix/FC8_linear_dresses.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
fc6_tops_embed = tsne.fit_transform(fc6_tops)

In [ ]:
plot_tsne(500,fc6_tops_embed,tops.iloc[subset_tops],'output/DF_tops_FC6_10000_tsne.png')

In [ ]:
fc8_relu_tops = np.where(fc8_tops>0,fc8_tops,0)
fc8_relu_tops_embed = tsne.fit_transform(fc8_relu_tops)

In [ ]:
plot_tsne(1000,fc8_relu_tops_embed,tops.iloc[subset_tops],'output/DF_tops_FC8_Relu_10000_tsne.png')

In [ ]:
from keras.layers import Dense

model = VGG16(weights='imagenet')

FC8 = Dense(1000, activation=None, name='FC8')
FC8_linear = Model(inputs = model.input,
                   output = FC8(model.get_layer('predictions').input))
FC8.set_weights(model.get_layer('predictions').get_weights())


In [ ]:
import time
import pickle
start = time.time()
fc8_tops = FC8_linear.predict(proc_tops)
pickle.dump(fc8_tops,open('Data/feature_matrix/FC8_linear_tops_loaded_weights.p','wb'))
print(f'{time.time()-start:.2f} s')

In [ ]:
model = VGG16(weights='imagenet')
FC7 = Model(inputs = model.input, output = model.get_layer('fc2').output)

In [ ]:
start = time.time()
fc7_relu_tops = FC7.predict(proc_tops)
pickle.dump(fc7_relu_tops,open('Data/feature_matrix/FC7_relu_tops.p','wb'))
print(f'{time.time()-start:.2f} s')